In [ ]:
import folium
import numpy as np
import pandas as pd
from scipy import sparse
import re

In [12]:
def generate_routes_dict(file_path: str):
   t = pd.read_csv(file_path)
   rte = {}
   typ = {}
   airport = {}
   for index, row in t.iterrows():
      wpt = row['waypoints']
        
      regex = re.compile(r'\((.*?)\)')
      temp = regex.findall(wpt)
      res = [tuple(sub.split(', ')) for sub in temp]
      route = []
      for a in res:
         xy = (float(a[0]), float(a[1]))
         route.append(xy)
      rte[row['route_id']] = route
      typ[row['route_id']] = row['route_type']
      airport[row['route_id']] = row['airport']
   return rte, typ, airport


In [13]:
def plot_routes(rte, typ) -> folium.Map:
    fmap = folium.Map(location=[35, -100], zoom_start=6)
    wpt = []
    for key in rte:
       if typ[key] == "ARRIVAL":
         continue
       if airport[key] != "KIAH":
         continue
       folium.PolyLine(rte[key], color="red", weight=1, opacity=1).add_to(fmap)
       folium.Marker(location=rte[key][1], popup =  key + "=" + typ[key] + "\n").add_to(fmap)
    return fmap
 
rte, typ, airport = generate_routes_dict("drive/MyDrive/aigames_data/route_definitions.csv")
plot_routes(rte, typ)

In [ ]:
t = pd.read_csv("drive/MyDrive/aigames_data/train_obs_avail_merged.csv")
import folium.plugins

from folium.features import *
class DivIcon(MacroElement):
    def __init__(self, html='', size=(30,30), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)

def plot_routes_status() -> folium.Map:
  fmap = folium.Map(location=[35, -100], zoom_start=6)
  logs = {}
  for index, row in t.iterrows():
    date = row['timestamp'][:10]
    hour = int(row['timestamp'][11:13])
    time = row['timestamp'][11:16]
    route_id = row['route_id']

    if date != "2021-05-08":
      continue

    open = row["status"] == 1.0
    if open:
      folium.Circle(rte[route_id][-1], 10000, color = "green").add_to(fmap)
    folium.PolyLine(rte[route_id], color="green" if open else "red", weight=2 if open else 6, opacity=1).add_to(fmap)
    if route_id not in logs:
      logs[route_id] = ""
    logs[route_id] += time + ("_open" if open else "_closed") + "\n"
  for key in logs:
    folium.Marker(location=rte[key][-1] if typ[key] == "DEPARTURE" else rte[key][0], popup =  key + "=" + typ[key] + "\n" + logs[key]).add_to(fmap)

  return fmap

plot_routes_status()

In [33]:
t = pd.read_csv("drive/MyDrive/aigames_data/route_definitions_without_duplicates.csv")
print(t)

     Unnamed: 0      route_id airport route_type  \
0             0           TTT    KDFW    ARRIVAL   
1             1           TTT    KDFW  DEPARTURE   
2             2         BOOVE    KDFW    ARRIVAL   
3             3         CLARE    KDFW  DEPARTURE   
4             4      CQY CQY9    KDFW    ARRIVAL   
..          ...           ...     ...        ...   
229         236  HAROB6 HISKU    KSEA  DEPARTURE   
230         237  JEFPO1 JEFPO    KSEA  DEPARTURE   
231         238  MONTN1 ZADON    KSEA  DEPARTURE   
232         239  OZWLD1 OZWLD    KSEA  DEPARTURE   
233         240  SUMMA2 SUMMA    KSEA  DEPARTURE   

                                             waypoints  
0    [(32.86916111111111, -97.04050277777777), (32....  
1    [(32.897333333333336, -97.03769444444444), (32...  
2    [(32.382600000000004, -97.62483888888889), (32...  
3    [(32.897333333333336, -97.03769444444444), (32...  
4    [(32.18572222222222, -96.21810277777777), (32....  
..                               

In [34]:
t['direction'] = 0
import math


def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


def plot_routes(rte, typ) -> folium.Map:
    fmap = folium.Map(location=[35, -100], zoom_start=6)

    for index, row in t.iterrows():
      wpt = row['waypoints']
      id = row['route_id']
      regex = re.compile(r'\((.*?)\)')
      temp = regex.findall(wpt)
      res = [tuple(sub.split(', ')) for sub in temp]
      route = []
      for a in res:
        xy = (float(a[0]), float(a[1]))
        route.append(xy)
      p1 = route[0] if typ[id] == "DEPARTURE" else route[-2]
      p2 = route[1] if typ[id] == "DEPARTURE" else route[-1]
      dy = p2[0] - p1[0]
      dx = p2[1] - p1[1]
      folium.PolyLine([p1, p2], color="red", weight=1, opacity=1).add_to(fmap)
      #angle = 180.0 / math.pi * math.atan(dx / dy)
      angle = 180.0 / math.pi * angle_between((0, 1), (dx, dy))
      if dx < 0:
        angle = 360.0 - angle
      #if angle < 0:
      #  angle += 360
      folium.Marker(location=rte[id][1] if typ[id] == "DEPARTURE" else rte[id][-2], popup =  id + "=" + typ[id] + "\n" + str(angle)).add_to(fmap)
      t['direction'][index] = int(angle)
    return fmap

plot_routes(rte, typ)
print(t)
t.to_csv("drive/MyDrive/aigames_data/route_definitions_with_directions.csv")


     Unnamed: 0      route_id airport route_type  \
0             0           TTT    KDFW    ARRIVAL   
1             1           TTT    KDFW  DEPARTURE   
2             2         BOOVE    KDFW    ARRIVAL   
3             3         CLARE    KDFW  DEPARTURE   
4             4      CQY CQY9    KDFW    ARRIVAL   
..          ...           ...     ...        ...   
229         236  HAROB6 HISKU    KSEA  DEPARTURE   
230         237  JEFPO1 JEFPO    KSEA  DEPARTURE   
231         238  MONTN1 ZADON    KSEA  DEPARTURE   
232         239  OZWLD1 OZWLD    KSEA  DEPARTURE   
233         240  SUMMA2 SUMMA    KSEA  DEPARTURE   

                                             waypoints  direction  
0    [(32.86916111111111, -97.04050277777777), (32....          5  
1    [(32.897333333333336, -97.03769444444444), (32...        185  
2    [(32.382600000000004, -97.62483888888889), (32...         48  
3    [(32.897333333333336, -97.03769444444444), (32...        100  
4    [(32.18572222222222, -96.21810

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
